In [1]:
import numpy as np
from numpy.core.fromnumeric import shape, size
from random import random
from keras.datasets import mnist

Stand_Alone functions


In [3]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def rand(row, column):
    returned = np.empty((row, column))
    for j in range(row):
        rando=[]
        for i in range(column):
            rando.append(random())
            i+=1
        returned[j] = rando
    return returned


def Normalize(X):
    return((X-X.min())/(X.max()-X.min()))

def Binarize(X):
    X [X < 100] = 0
    X [X >= 100] = 1       
    return X

def labelVector (num):
    n = np.empty((len(num), 10))
    for i in range (len(num)):
        re = np.zeros((1 , 10))
        re[:,num[i]] = 1
        n[i] =re
    return n

def getMax(ma):
    for i in range(shape(ma)[0]):
        index = 0
        for j in range((shape(ma)[1])-1):
            if(ma[i][j+1]>ma[i][index]):
                index = j+1       
        for l in range(shape(ma)[1]):
            if(l==index):
                ma[i][l] =1 
            else:
                ma[i][l] =0      
    return ma


def accuracy(predicted_y, y):
    guess_wrong=0
    for i in range(len(predicted_y)):
        if(not np.array_equiv(predicted_y[i],y[i])):
            guess_wrong+=1
    acc = ((len(predicted_y)- guess_wrong) / len(predicted_y))
    return (acc*100)

Neural Networks Architecture 


In [4]:
class Layer:
    nerweight =0
    inputs = 0
    outputs=0
    delta=0
    def __init__(self, neuronNumber,length):
        self.nerweight =rand(neuronNumber, length)
        #self.bais = rand(neuronNumber , 1)

    def feedForward(self, inputs):
        self.outputs= []
        self.inputs = inputs
        self.outputs  = sigmoid(np.dot(np.array(self.nerweight), self.inputs))
        return self.outputs 

    
    def fit(self, eta, target, outj, sumDelta):
        if(size(sumDelta)==0):
            self.delta=(target-self.outputs)*self.outputs*(1-self.outputs)
            dw = eta*np.transpose([self.delta])*np.transpose(outj)
            self.nerweight = self.nerweight + dw 
            return np.dot(self.delta, self.nerweight)
        else:
            self.delta=sumDelta*self.outputs*(1-self.outputs)
            dw = eta*self.delta*np.transpose([outj])
            self.nerweight = self.nerweight + np.transpose(dw)
            return np.dot(self.delta, self.nerweight)
            

In [36]:
class Network:
    layers = 0
    layernum=0
    length=0
    orin=0
    inputs =0
    target =0
    def __init__(self, NumNeuron, inputs, target):
        self.layers = []
        self.orin = inputs
        self.inputs =inputs
        self.target = target
        length = shape(inputs)[1]
        #One hidden layer layer
        if(len(NumNeuron) ==0):
            layer = Layer(4, length)  
            length =4
            self.layers.append(layer)
        else: 
            for i in range (len(NumNeuron)):
                if(NumNeuron[i]<4):
                    NumNeuron[i] = 4
                layer = Layer(NumNeuron[i], length)  
                length = NumNeuron[i]   
                self.layers.append(layer)
                
        #default Output layer
        layer = Layer(shape(target)[1], length)  
        self.layers.append(layer)

    def feedForward(self, data):
        current =np.copy(data)
        for i in range(len(self.layers)):
            current= self.layers[i].feedForward(current)
        return current
    
    def fit(self, eta, leranRate):
        for j in range(leranRate):
            now = np.empty((shape(self.target)))
            for k in range((shape(self.orin)[0])): 
                now[k] =self.feedForward(self.orin[k])   
                sumdelta = []
                for i in range(len(self.layers)-1, 0, -1):
                    sumdelta =self.layers[i].fit(eta, self.target[k], self.layers[i-1].outputs, sumdelta)
                sumdelta =self.layers[0].fit(eta, self.target[k], self.orin[k], sumdelta)
            if(j%10 == 0):
                print (accuracy(getMax(now), train_y))


Image Processing Functions 


In [6]:
def centroid(X):
    M00=np.sum(X.sum(axis=1))
    M10 =np.array([])
    M01 =np.array([])
    for i in range(X.shape[0]):
        M10= np.append(M10, (X[i].sum(axis=0)*(i+1)))
        
    for i in range(X.shape[1]):
        M01 = np.append(M01,(X[:,i].sum(axis=0)*(i+1)))
    M10 = np.sum(M10)
    M01 = np.sum(M01)
    if(M00==0):
        Xc= 0
        Yc= 0
    else:
        Xc = M10/M00
        Yc = M01/M00
    return np.array([Xc, Yc])

In [7]:
def FeatureImage(X):
    vector=np.array([])
    for i in range(2):
        for j in range(4):              #0-7, 0--14  
            vector= np.append(vector, centroid(X[j*7:((j+1)*7) ,i*14:((i+1)*14)])) 
    return vector

Application

In [8]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()


In [ ]:
train_X =Binarize(train_X)
test_X = Binarize(test_X)

In [9]:
vec = np.array([])
for i in range(60000):
    vec = np.append(vec, FeatureImage(train_X[i]))
vec.shape

(960000,)

In [10]:
vec =vec.reshape(60000, 16)
vec.shape

(60000, 16)

In [11]:
vecTest = np.array([])
for i in range(10000):
    vecTest = np.append(vecTest, FeatureImage(test_X[i]))
print(vecTest.shape)
vecTest =vecTest.reshape(10000, 16)
print(vecTest.shape)

(160000,)
(10000, 16)


In [12]:
test_y = labelVector(test_y)


In [16]:
train_y =labelVector(train_y)

TEST

In [45]:
Test1 = Network([4, 6], vec[0:20000], train_y[0:20000])
Test1.fit(0.03, 100)
preY1 = np.empty((shape(test_y)))
for i in range(len(test_y)-5000):
    preY1[i]=Test1.feedForward(vecTest[i])
print(accuracy(getMax(preY1), test_y))

10.345
10.395
10.405000000000001
10.435
10.43
10.45
10.445
10.405000000000001
10.41
10.4
9.46


In [47]:
Test2 = Network([4, 6], vec[0:20000], train_y[0:20000])
Test2.fit(0.05, 100)
preY2 = np.empty((shape(test_y)))
for i in range(len(test_y)-5000):
    preY2[i]=Test2.feedForward(vecTest[i])
print(accuracy(getMax(preY2), test_y))

10.355
10.31
10.295
10.290000000000001
10.295
10.315000000000001
10.255
10.18
10.17
10.12
9.8


In [48]:
Test3 = Network([15, 8, 20], vec[0:20000], train_y[0:20000])
Test3.fit(0.01, 100)
preY3 = np.empty((shape(test_y)))
for i in range(len(test_y)-5000):
    preY3[i]=Test3.feedForward(vecTest[i])
print(accuracy(getMax(preY3), test_y))

9.855
9.855
9.855
9.855
10.334999999999999
10.375
10.375
10.375
10.365
10.365
9.8


In [49]:
Test4 = Network([15, 8, 20], vec[0:40000], train_y[0:40000])
Test4.fit(0.03, 100)
preY4 = np.empty((shape(test_y)))
for i in range(len(test_y)-5000):
    preY4[i]=Test4.feedForward(vecTest[i])
print(accuracy(getMax(preY4), test_y))

9.65
9.525
9.47
9.4125
9.41
9.417499999999999
9.415
9.4125
9.4125
9.417499999999999
10.91


In [50]:
Test5 = Network([15, 8, 20], vec[0:40000], train_y[0:40000])
Test5.fit(0.05, 25)
Test5.fit(0.03, 50)
Test5.fit(0.01, 25)
preY5 = np.empty((shape(test_y)))
for i in range(len(test_y)-5000):
    preY5[i]=Test5.feedForward(vecTest[i])
print(accuracy(getMax(preY5), test_y))

11.407499999999999
9.292499999999999
9.305
9.415
9.41
9.417499999999999
9.415
9.415
10.3475
10.37
10.37
10.91


In [51]:
Test6 = Network([15, 8, 20], vec[0:40000], train_y[0:40000])
Test6.fit(0.05, 25)
Test6.fit(0.03, 50)
Test6.fit(0.01, 25)


9.81
9.3125
9.3025
9.41
9.417499999999999
9.417499999999999
9.415
9.41
10.342500000000001
10.365
10.3675


TypeError: getMax() got an unexpected keyword argument 'preY5'

In [52]:
preY6 = np.empty((shape(test_y)))
for i in range(len(test_y)-5000):
    preY6[i]=Test6.feedForward(vecTest[i])
print(accuracy(getMax(preY6), test_y))

10.91
